In [1]:
import findspark
findspark.init()
findspark.find()

'c:\\Users\\ammar\\anaconda3\\Lib\\site-packages\\pyspark'

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc=SparkContext.getOrCreate()
spark=SparkSession(sc)
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
# write a pyspark code to calculate the sum of all xalues in an RDD) using an accumulator. Create an RDD
# of integers from 1 to 10. Define an accutnulator vsith an initial value of 0 use the foreach
# action on the RDD to update the accumulator with each element. Finally print the final value
# of the accumulator after processing all elements in the RDD.

integer_rdd = sc.parallelize(range(1, 11))
accum = sc.accumulator(0)
print(integer_rdd.collect())
integer_rdd.foreach(lambda x: accum.add(x))
print(accum.value)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
55


In [ ]:
# Inner Join: Perform an inner join on emp_id between rddl and rdd2 to get an RDD containing the
# employee ID. employee name, employee department. and employee salary for employees who exist In
# both RDDs.
# Left Outer Join: Perform a left outer join on emp_id between rddl and rdd2 to get an RDD containing
# the employee ID. employee name, employee department, and employee salary for all employees jn
# rddl and matching employees in rdd2. For employees in rddl that do not have a matching employee
# ID in rdd2, the employee salary should be set to None.
# Calculate the total salary of employees in RDDI by joining it with RDD2 on "emp_id" and
# multiplying "emp_salary" with "emp_experience".
# Calculate the maximum salary of employees
# Perform an inner join on RDDI and RDD2 on "emp_id" and "dept_id". and calculate the total salary
# of employees in each department.
# Perform a left outer join on RDDI and RDD2 on "emp_id" and "dept_id", and find the employees
# who do not belong to anv department.
# CODE IS CONCEPTUAL

rddl = sc.parallelize([(1, 'John', 25), (2, 'Smith', 22), (3, 'Williams', 32), (4, 'Chris', 23)])
rdd2 = sc.parallelize([(1, 'HR', 5), (2, 'Finance', 3), (3, 'IT', 4), (5, 'Admin', 2)])


# Inner Join
inner_join_rdd = rddl.join(rdd2, on='emp_id', how='inner')
inner_join_rdd.select('emp_id', 'emp_name', 'emp_department', 'emp_salary').show()

# Left Outer Join
left_outer_join_rdd = rddl.join(rdd2, on='emp_id', how='left_outer')
left_outer_join_rdd.select('emp_id', 'emp_name', 'emp_department', 'emp_salary').show()

# Calculate total salary
total_salary_rdd = rddl.join(rdd2, on='emp_id', how='inner')
total_salary_rdd = total_salary_rdd.withColumn('total_salary', total_salary_rdd['emp_salary'] * total_salary_rdd['emp_experience'])
total_salary = total_salary_rdd.selectExpr('sum(total_salary) as total_salary').collect()[0]['total_salary']
print(total_salary)

# Calculate maximum salary
max_salary = rddl.selectExpr('max(emp_salary) as max_salary').collect()[0]['max_salary']
print(max_salary)

# Calculate total salary by department
total_salary_by_dept = rddl.join(rdd2, (rddl.emp_id == rdd2.emp_id) & (rddl.dept_id == rdd2.dept_id), how='inner')
total_salary_by_dept = total_salary_by_dept.groupBy('emp_department').agg({'emp_salary': 'sum'})
total_salary_by_dept.show()

# Employees not belonging to any department
no_dept_employees = rddl.join(rdd2, (rddl.emp_id == rdd2.emp_id) & (rddl.dept_id == rdd2.dept_id), how='left_anti')
no_dept_employees.show()


In [ ]:
from pyspark.sql.types import StringType

data = ["Project Gutenberg's", "Alice's Adventures in Wonderland", "Project Gutenberg's", "Adventures in Wonderland", "Project Gutenberg's"]

df = spark.createDataFrame(data, StringType()).toDF('data')
df.show()


+--------------------+
|                data|
+--------------------+
| Project Gutenberg's|
|Alice's Adventure...|
| Project Gutenberg's|
|Adventures in Won...|
| Project Gutenberg's|
+--------------------+



In [ ]:
rdd = sc.parallelize(data)
rdd = rdd.map(lambda x: x.split(" "))
rdd.collect()

[['Project', "Gutenberg's"],
 ["Alice's", 'Adventures', 'in', 'Wonderland'],
 ['Project', "Gutenberg's"],
 ['Adventures', 'in', 'Wonderland'],
 ['Project', "Gutenberg's"]]

In [ ]:
# Apply flatMap transformation and split the strings
rdd_flatmap = sc.parallelize(data)
rdd_flatmap = rdd_flatmap.flatMap(lambda x: x.split(" "))
rdd_flatmap.collect()

['Project',
 "Gutenberg's",
 "Alice's",
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 "Gutenberg's",
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 "Gutenberg's"]

In [ ]:
def split_words(line):
    return line.split(" ")
def make_pairs(word):
    return (word, 1)

rdd = sc.parallelize(data)
pair_rdds = rdd.flatMap(split_words).map(make_pairs)
pair_rdds.collect()

[('Project', 1),
 ("Gutenberg's", 1),
 ("Alice's", 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ("Gutenberg's", 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ("Gutenberg's", 1)]

In [ ]:
# For example, Suppose RDD contains first five natural numbers (1, 2, 3, 4, and 5) and the
# predicate is check for an even number. The resulting RDD after the filter will contain only
# the even numbers i.e., 2 and 4.
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

In [ ]:
%%writefile people.csv
name;age;job
Jorge;30;Developer
Bob;32;Developer

Writing people.csv


In [ ]:
peopledf = spark.read.csv('people.csv', sep=';', header=True,inferSchema=True)
peopledf.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [ ]:
peopledf.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)



In [ ]:
peopledf.select('name').show()

+-----+
| name|
+-----+
|Jorge|
|  Bob|
+-----+



In [ ]:
peopledf.select('name', peopledf.age+1).show()

+-----+---------+
| name|(age + 1)|
+-----+---------+
|Jorge|       31|
|  Bob|       33|
+-----+---------+



In [ ]:
# filter on multiple conditions
peopledf.filter((peopledf.age > 30) & (peopledf.job == 'Developer')).show()

+----+---+---------+
|name|age|      job|
+----+---+---------+
| Bob| 32|Developer|
+----+---+---------+



In [ ]:
peopledf.groupBy('age').count().show()

+---+-----+
|age|count|
+---+-----+
| 32|    1|
| 30|    1|
+---+-----+



sparksql

In [ ]:
peopledf.createOrReplaceTempView('people')
spark.sql('SELECT * FROM people').show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [ ]:
data = ['great first try super boy', 'first try good', 'super boy won first try']
rdd = sc.parallelize(data)
pairs = rdd.map(lambda x: (x.split(" ")[0], x))
# • Creating a pair RDD using the first word as the key in Python
pairs.collect()

[('great', 'great first try super boy'),
 ('first', 'first try good'),
 ('super', 'super boy won first try')]

In [ ]:
result = pairs.filter(lambda Value: len(Value[1]) < 15)
# • Simple filter on second element in Pytho
result.collect()

[('first', 'first try good')]

pair rdds

Reduce-like operations are transformations that combine values associated with the same key across partitions. They are commonly used for aggregation tasks, such as counting, summing, or finding the minimum or maximum value.

reduceByKey

Takes an RDD of key-value pairs and returns a new RDD with the same keys, but with the values reduced to a single value using a specified reduce function.
The reduce function takes two values of the same type and returns a single value of the same type.
Example: To sum the values associated with each key in an RDD:

In [ ]:
rdd = sc.parallelize([["a", 3], ["b", 4], ["a", 1]])
rk = rdd.reduceByKey(lambda x,y: x+y)
rk.collect()

[('a', 4), ('b', 4)]

groupByKey

Takes an RDD of key-value pairs and returns a new RDD with the same keys, but with the values grouped into a single list for each key.
Example: To group the values associated with each key in an RDD:

```• rdd = sc.parallelize([["a", 3], ["b", 4], ["a", 1]])
• rk = rdd.reduceByKey(lambda x,y: x+y)
• (‘a’,4)
• (‘b’,4)
• rdd.groupByKey()
• (‘a’, (3, 1))
• (‘b’, (4))
```

sort
```
• rdd = sc.parallelize([["a", 3],
["b", 4], ["a", 1]])
• rdd1.sortByKey()
[‘a’ , 3 ]
[‘a’ , 1 ]
[‘b’ , 4 ]
```

In [ ]:
Data = sc.parallelize(["Spark is faster than Hadoop", "Spark is easy to use"]) #rdds
words = Data.flatMap(lambda line: line.split(" "))#rdds #transformations
wordmap = words.map(lambda word:(word,1))#rdds #transformations
wordcount = wordmap.reduceByKey(lambda x,y: x+y)#rdds #transformations
wordcount.collect()#action

[('to', 1),
 ('easy', 1),
 ('is', 2),
 ('Hadoop', 1),
 ('faster', 1),
 ('than', 1),
 ('use', 1),
 ('Spark', 2)]

In [ ]:
rdd = sc.parallelize([["a", 3], ["b", 4], ["a", 1]])
# mapValues
rdd=rdd.mapValues(lambda x: x+2)
# • Apply a function to each value of a pair RDD without
# changing the key
rdd.collect()

[('a', 5), ('b', 6), ('a', 3)]

In [ ]:
rdd=sc.parallelize([("a", 3), ("b", 4), ("a", 1)])
rdd=rdd.flatMapValues(lambda x: range(0,x) )
rdd.collect()

[('a', 0),
 ('a', 1),
 ('a', 2),
 ('b', 0),
 ('b', 1),
 ('b', 2),
 ('b', 3),
 ('a', 0)]

In [ ]:
# You are given a file containing marks obtained by each student in different
# assessments in the form (marks, StudentNo)
data = sc.parallelize([(8,'S1'),(9,'S2'),(7,'S1'),(9,'S2'),(8,'S1'),(5,'S2')])
# 1. Find the average marks obtained by each student
rdd = data.map(lambda x: (x[1], x[0]))
rdd2=rdd.mapValues(lambda x: (x,1))
rdd3=rdd2.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
avg=rdd3.mapValues(lambda x: x[0]/x[1])
avg.collect()


[('S2', 7.666666666666667), ('S1', 7.666666666666667)]

working<br>
('S1', (8, 1))<br> ('S2', (2, 1))<br> ('S3', (7, 1))<br> ('S1', (4, 1))<br> ('S2', (5, 1))<br>
('S1', (2, 1))<br>
('S4', (5, 1))<br> reducebykey<br>
('S1', (14, 3)) <br>('S2', (7, 2))<br> ('S3', (7, 1))<br> ('S4', (5, 1)) <br>avgbykey<br> ('S1', 4.666666666666<br>
('S2', 3.5)<br>
('S3', 7.0)<br>
('54' 5.0))

In [ ]:
# find average marks of assessments
data = sc.parallelize([(8,'S1'),(2,'S2'),(7,'S3'),(4,'S1'),(5,'S2'),(2,'S1'),(5,'S4')])
rddtotal=data.map(lambda x: (x[0]))
t=rddtotal.reduce(lambda x,y: x+y)
total=data.map(lambda x: (x[0])).sum()
print(total/data.count())

4.714285714285714


In [ ]:
# Join-like Operations Examples
rdd1 = sc.parallelize([["a", 3], ["b", 4], ["a", 1]])
rdd2= sc.parallelize([["a",5],["c",10]])
rdd1.join(rdd2).collect()

[('a', (3, 5)), ('a', (1, 5))]

In [ ]:
rdd1.leftOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (3, 5)), ('a', (1, 5))]

In [ ]:
rdd1.rightOuterJoin(rdd2).collect()

[('c', (None, 10)), ('a', (3, 5)), ('a', (1, 5))]

In [ ]:
[(x, tuple(map(list, y))) for x, y in sorted(list(rdd1.cogroup(rdd2).collect()))]
# • Group data from both RDDs sharing the same key.
# • It’s a full outer join.

[('a', ([3, 1], [5])), ('b', ([4], [])), ('c', ([], [10]))]

Finals prep

In [8]:
# sales_transactions table
sales_transactions = [
    {"transaction_id": 1, "customer_id": 1, "product_id": 1, "quantity": 2, "transaction_date": "2024-01-01"},
    {"transaction_id": 2, "customer_id": 1, "product_id": 2, "quantity": 1, "transaction_date": "2024-01-02"},
    {"transaction_id": 3, "customer_id": 2, "product_id": 3, "quantity": 3, "transaction_date": "2024-01-01"},
    {"transaction_id": 4, "customer_id": 2, "product_id": 4, "quantity": 2, "transaction_date": "2024-01-03"},
]

customers = [
    {"customer_id": 1, "customer_name": "John", "customer_email": "johndoe@company", "customer_age": 30, "customer_city": "New York"},
    {"customer_id": 2, "customer_name": "Jane", "customer_email": "janedoe@company", "customer_age": 25, "customer_city": "Los Angeles"},
    {"customer_id": 3, "customer_name": "Bob", "customer_email": "bob@company", "customer_age": 40, "customer_city": "Chicago"},
    {"customer_id": 4, "customer_name": "Alice", "customer_email": "alice@company", "customer_age": 35, "customer_city": "Houston"},
]

products = [
    {"product_id": 1, "product_name": "Product 1", "product_category": "Category 1", "product_price": 10.99},
    {"product_id": 2, "product_name": "Product 2", "product_category": "Category 2", "product_price": 19.99},
    {"product_id": 3, "product_name": "Product 3", "product_category": "Category 1", "product_price": 8.99},
    {"product_id": 4, "product_name": "Product 4", "product_category": "Category 3", "product_price": 29.99},
]

In [9]:
# Suppose you're tasked with analyzing the sales performance of a retail company You have been provided
# a dataset containing information about sales transactions, customer details and product information. The dataset
# consists of multiple tables
# 1.sales _ transactions table:
# Columns: transaction_id, customer_id, product_id, quantity, transaction_date.
# 2. customers table:
# Columns: customer_id, customer name, customer_email, customer_age, customer_city.
# 3. products table:
# Columns: product_id, product _ name, product _ category, product _ price.

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Initialize Spark
spark = SparkSession.builder.master("local").appName("SQL on Dictionaries").getOrCreate()

# Define schema for each table
sales_schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("transaction_date", StringType(), True)
])

customer_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("customer_email", StringType(), True),
    StructField("customer_age", IntegerType(), True),
    StructField("customer_city", StringType(), True)
])

product_schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("product_name", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("product_price", FloatType(), True)
])

# Creating Spark DataFrames with explicit schema
sales_sdf = spark.createDataFrame(sales_transactions, schema=sales_schema)
customers_sdf = spark.createDataFrame(customers, schema=customer_schema)
products_sdf = spark.createDataFrame(products, schema=product_schema)

# Register DataFrames as temporary views to run SQL queries
sales_sdf.createOrReplaceTempView("sales_transactions")
customers_sdf.createOrReplaceTempView("customers")
products_sdf.createOrReplaceTempView("products")

# Example SQL-like operation using Spark SQL
query = """
SELECT c.customer_name, SUM(s.quantity * p.product_price) AS total_spent
FROM sales_transactions s
JOIN customers c ON s.customer_id = c.customer_id
JOIN products p ON s.product_id = p.product_id
GROUP BY c.customer_name
"""

result_sdf = spark.sql(query)
result_sdf.show()

+-------------+-----------------+
|customer_name|      total_spent|
+-------------+-----------------+
|         John|41.96999931335449|
|         Jane|86.94999885559082|
+-------------+-----------------+



In [10]:
# Write a SQL query to find the total sales revenue generated by each product category for the current year.
# query should return the product category along with the total revenue. Assume the current year is 2024
# also assume the tables are like the tempviews you have made

query ="""
SELECT p.product_category, SUM(p.product_price) as total_revenue
FROM products p
JOIN sales_transactions s on p.product_id=s.product_id
WHERE YEAR(transaction_date) = 2024
GROUP BY p.product_category
"""

result_sdf = spark.sql(query)
result_sdf.show()

+----------------+------------------+
|product_category|     total_revenue|
+----------------+------------------+
|      Category 1|19.979999542236328|
|      Category 3|29.989999771118164|
|      Category 2|19.989999771118164|
+----------------+------------------+



In [13]:
# Now, load the provided dataset into Python using Pandas DataFrames.
# Write Python code to:
# i. Find the top 5 customers who made the highest total purchase amount.
# ii. Calculate the average age of customers who made purchases in the Electronics category

import pandas as pd

# Define the data as provided earlier
sales_transactions = [
    {"transaction_id": 1, "customer_id": 1, "product_id": 1, "quantity": 2, "transaction_date": "2022-01-01"},
    {"transaction_id": 2, "customer_id": 1, "product_id": 2, "quantity": 1, "transaction_date": "2022-01-02"},
    {"transaction_id": 3, "customer_id": 2, "product_id": 3, "quantity": 3, "transaction_date": "2022-01-01"},
    {"transaction_id": 4, "customer_id": 2, "product_id": 4, "quantity": 2, "transaction_date": "2022-01-03"},
]

customers = [
    {"customer_id": 1, "customer_name": "John", "customer_email": "johndoe@company.com", "customer_age": 30, "customer_city": "New York"},
    {"customer_id": 2, "customer_name": "Jane", "customer_email": "janedoe@company.com", "customer_age": 25, "customer_city": "Los Angeles"},
    {"customer_id": 3, "customer_name": "Bob", "customer_email": "bob@company.com", "customer_age": 40, "customer_city": "Chicago"},
    {"customer_id": 4, "customer_name": "Alice", "customer_email": "alice@company.com", "customer_age": 35, "customer_city": "Houston"},
]

products = [
    {"product_id": 1, "product_name": "Laptop", "product_category": "Electronics", "product_price": 1000.0},
    {"product_id": 2, "product_name": "Smartphone", "product_category": "Electronics", "product_price": 700.0},
    {"product_id": 3, "product_name": "Keyboard", "product_category": "Electronics", "product_price": 150.0},
    {"product_id": 4, "product_name": "Mouse", "product_category": "Electronics", "product_price": 50.0},
]

# Convert dictionaries to Pandas DataFrames
sales_df = pd.DataFrame(sales_transactions)
customers_df = pd.DataFrame(customers)
products_df = pd.DataFrame(products)

# i
# Merge the necessary DataFrames
merged_df = sales_df.merge(customers_df, on="customer_id").merge(products_df, on="product_id")

# Calculate the total spent per transaction
merged_df['total_spent'] = merged_df['quantity'] * merged_df['product_price']

# Aggregate to find the top 5 customers
top_customers = merged_df.groupby('customer_name').total_spent.sum().nlargest(5).reset_index()

print("Top 5 customers by total purchase amount:")
print(top_customers)

# ii
# Filter for the Electronics category
electronics_purchases = merged_df[merged_df['product_category'] == 'Electronics']

# Calculate the average age of customers who purchased electronics
average_age_electronics = electronics_purchases['customer_age'].mean()

print(f"Average age of customers who made purchases in the Electronics category: {average_age_electronics:.2f}")


Top 5 customers by total purchase amount:
  customer_name  total_spent
0          John       2700.0
1          Jane        550.0
Average age of customers who made purchases in the Electronics category: 27.50


In [14]:
# Assume you need to find the total sales quantity for each product category but only for the customers residing
# in 'New York'.
# i. Write an SQL query to filter out the transactions made b customers from 'New York' and calculate the total
# sales quantity for each product category
# ii. Now, using the filtered dataset obtained from the SQL query. write Python code using DataFrames to
# calculate the average product price for each product category

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("Sales Analysis").getOrCreate()

# Create DataFrames for sales, customers, and products
sales_df = spark.createDataFrame(sales_transactions)
customers_df = spark.createDataFrame(customers)
products_df = spark.createDataFrame(products)

# Register DataFrames as temporary views
sales_df.createOrReplaceTempView("sales_transactions")
customers_df.createOrReplaceTempView("customers")
products_df.createOrReplaceTempView("products")

# i
# Perform the SQL query using PySpark SQL
query = """
SELECT p.product_category, SUM(s.quantity) AS total_quantity
FROM sales_transactions s
JOIN customers c ON s.customer_id = c.customer_id
JOIN products p ON s.product_id = p.product_id
WHERE c.customer_city = 'New York'
GROUP BY p.product_category
"""
filtered_sales = spark.sql(query)

# Show the results of the SQL query
filtered_sales.show()


+----------------+--------------+
|product_category|total_quantity|
+----------------+--------------+
|     Electronics|             3|
+----------------+--------------+



In [18]:
# ii
# Join the filtered sales with the products to get pricing information
category_price_df = filtered_sales.join(products_df, filtered_sales.product_category == products_df.product_category)

# Calculate the average product price per category
average_price = category_price_df.groupBy("p.product_category").agg(avg("product_price").alias("average_price"))

# Show the average prices
average_price.show()


+----------------+-------------+
|product_category|average_price|
+----------------+-------------+
|     Electronics|        475.0|
+----------------+-------------+



In [ ]:
# We have a large dataset containing information about students' grades in different
# subjects at a university. The dataset is structured as follows: each record consists of the student's ID,
# subject code, and grade obtained in that subject. Consider an input file in the following format:


In [2]:
%%writefile grades.txt
S001 Math A
S002 English B
S003 Science C
S001 Science B
S002 Math A
S003 English B
S004 Math C
S004 English A
S005 Science A
S005 Math B

Writing grades.txt


In [4]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("Grades Analysis").getOrCreate()

# Read the input file
df = spark.read.text("grades.txt")

# Split the line into separate columns
df = df.selectExpr("split(value, ' ')[0] as StudentID",
                   "split(value, ' ')[1] as Subject",
                   "split(value, ' ')[2] as Grade")

# Task i: Select and display the records of students who have obtained grade 'A' in at least one subject.
students_with_a = df.filter(df.Grade == 'A')
students_with_a.show()
from pyspark.sql.functions import col, when, avg

# Task ii: Calculate the average grade obtained by each student across all subjects. 
# Display the student ID along with their average grade.
# We need to convert grades to numerical values, assuming A=4, B=3, C=2 for simplicity
grade_to_numeric = when(col("Grade") == 'A', 4).when(col("Grade") == 'B', 3).when(col("Grade") == 'C', 2)
df_with_numeric_grades = df.withColumn("NumericGrade", grade_to_numeric)

avg_grades = df_with_numeric_grades.groupBy("StudentID").agg(avg("NumericGrade").alias("AvgGrade"))
avg_grades.show()

# Task iii: Find the subject in which the highest number of students have obtained grade 'A'. 
# Display the subject code along with the count of students who received grade 'A' in that subject.
top_subject_for_a = students_with_a.groupBy("Subject").count().orderBy(col("count").desc()).first()
print(f"Subject with most A's: {top_subject_for_a['Subject']}, Count: {top_subject_for_a['count']}")

# Task iv: Sort the records based on the student ID in ascending order.
sorted_records = df.sort("StudentID")
sorted_records.show()

+---------+-------+-----+
|StudentID|Subject|Grade|
+---------+-------+-----+
|     S001|   Math|    A|
|     S002|   Math|    A|
|     S004|English|    A|
|     S005|Science|    A|
+---------+-------+-----+

+---------+--------+
|StudentID|AvgGrade|
+---------+--------+
|     S004|     3.0|
|     S001|     3.5|
|     S002|     3.5|
|     S005|     3.5|
|     S003|     2.5|
+---------+--------+

Subject with most A's: Math, Count: 2
+---------+-------+-----+
|StudentID|Subject|Grade|
+---------+-------+-----+
|     S001|   Math|    A|
|     S001|Science|    B|
|     S002|English|    B|
|     S002|   Math|    A|
|     S003|Science|    C|
|     S003|English|    B|
|     S004|English|    A|
|     S004|   Math|    C|
|     S005|Science|    A|
|     S005|   Math|    B|
+---------+-------+-----+

